In [5]:
# Small Language Model(SLLM) 

# Step 02 - 트랜스포머 허깅페이스 데이터셋(WikiText-2)을 적용하여 Small Language Model(SLLM) 구축 및 텍스트 생성
# 데이터셋 -> 데이터로더 -> GPU 설정 -> SLLM 모델 정의 -> 학습 -> 텍스트 생성 -> 테스트

# 핵심 문제 - 영어 기반이라 감성적인 한국어 여행 블로그를 생성하기엔 한계
# - 단어 반복 (“jenkin”) : WikiText-2에 자주 등장하는 고유명사
# - 문맥 불일치 : 프롬프트는 감성 여행, 데이터는 역사/스포츠
# - 의미 연결 부족 : 모델이 감성적 흐름을 학습하지 못함

# 기능 추가
# - 샘플링 방식(top-k, top-p, temperature) 적용: 다양성 확보
# - 프롬프트 튜닝: 감성적 블로그 스타일 유도
# - 후처리(clean_text): 출력 품질 향상

In [6]:
# import 및 GPU 설정
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset # HuggingFace Datasets

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Pytorch Version : {torch.__version__}, Device : {device}')

Pytorch Version : 2.7.1+cu118, Device : cuda


In [7]:
# WikiText-2 데이터셋 로딩 및 전처리
# - Step 02 에서는 WikiText-2 데이터셋를 사용해 토큰화하고, 시퀀시를 만들어 학습용으로 변환한다.

# Hugging Face에서 WikiText-2 로드 및 다운로드
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
# print('dataset : ', dataset)
# dataset :  Dataset({
#     features: ['text'],
#     num_rows: 36718
# })

# 간단한 토크나이저 사용 - 단어 단위
from collections import Counter

def tokenizer(text):
    return text.lower().split()

# 전체 텍스트 토큰화
tokens = []
for item in dataset:
    tokens.extend(tokenizer(item['text']))

# 단어 사전 생성
vocab = list(set(tokens)) # 분리된 단어 -> 리스트
# print(vacab)
word2idx = { word:idx for idx, word in enumerate(vocab) } # 단어 리스트 -> 딕셔너리(value:key), index 값 출력
# print(word2idx)
idx2word = { idx:word for word, idx in word2idx.items() } # 딕셔너리(key:value), word value 값 출력
# print(idx2word)

# 시퀀스 생성(입력: [32 단어], 출력: [32 단어] 등)
sequence_length = 32
data = []
for i in range(len(tokens) - sequence_length):
    input_seq = tokens[i:i+sequence_length]
    # print('input_seq : ', input_seq)
    target_seq = tokens[i+1:i+sequence_length+1]
    # print('target_seq : ', target_seq)
    data.append((
        torch.tensor([ word2idx[word] for word in input_seq ]),
        torch.tensor([ word2idx[word] for word in target_seq ]) 
    ))
# print(data[:1])

In [8]:
# 데이터셋 및 데이터로더 생성
class WikiTextDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# dataloader, 10000 -> 30000
dataloader = DataLoader(WikiTextDataset(data[:40000]), batch_size=8, shuffle=True)

In [9]:
# Transformer 기반 SLLM 모델 정의
class TransformerSLLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, max_len):
        super().__init__()

        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, vocab_size)
        
    def forward(self, x):
        seq_len = x.size(1)
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        x = self.token_embedding(x) + self.pos_embedding(positions)
        x = self.transformer(x)
        return self.fc_out(x)

In [10]:
# 학습
model = TransformerSLLM(
    vocab_size=len(vocab),
    embed_dim=128,
    num_heads=4,
    hidden_dim=256,
    num_layers=2,
    max_len=sequence_length
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs) # 모델 예측
        loss = criterion(outputs.view(-1, len(vocab)), targets.view(-1)) # 손실함수 값

        # 오차역전파
        optimizer.zero_grad() # 미분 파리미터 초기화
        loss.backward() # 미분 연산
        optimizer.step() # 미분 연산 후 가중치,바이어스 파라미터 업데이트

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

c:\Users\AI\miniconda3\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 20756.0720
Epoch 2, Loss: 16599.7573
Epoch 3, Loss: 16224.6203
Epoch 4, Loss: 16035.5944
Epoch 5, Loss: 15886.9274


In [11]:
# 텍스트 생성 함수 추가
# - 이 함수는 start_text를 기반으로 최대 50개의 단어를 이어서 생성합니다. 단어 사전에 없는 경우는 0으로 처리되며, 
# 학습된 문맥을 따라 다음 단어를 예측합니다.
# 샘플링 방식 추가 (top-k, top-p, temperature) - temperature=0.8 정도로 설정하면 더 다양하고 자연스러운 문장이 나올수 있음
# def generate_text(model, start_text, word2idx, idx2word, max_length=50, temperature=0.7):
def generate_text(model, start_text, word2idx, idx2word, max_length=50, temperature=0.7, top_k=50, top_p=0.9):
    model.eval()
    tokens = start_text.lower().split()
    input_ids = [ word2idx.get(word, 0) for word in tokens ] # unknown word -> 0

    input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)

    for _ in range(max_length):
        with torch.no_grad():
            output = model(input_tensor)
            next_token_logits = output[:, -1, :] / temperature # 마지막 위치의 출력
            logits = top_k_filtering(next_token_logits, top_k)
            logits = top_p_filtering(logits, top_p)
            probabilities = torch.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probabilities, num_samples=1)
            # next_token_id = torch.argmax(next_token_logits, dim=-1).item() # argmax 방식으로 가장 확률 높은 단어만 선택
            input_ids.append(next_token_id)

            input_tensor = torch.tensor(input_ids[-sequence_length:], dtype=torch.long).unsqueeze(0).to(device)

    # 텐서를 정수로 변환 후 단어로 매핑
    generated_words = [idx2word.get(int(idx), "[UNK]") for idx in input_ids]
    return ' '.join(generated_words)

# 현재는 torch.multinomial()로 확률 기반 샘플링을 하고 있지만, 더 정교한 제어를 위해 top-k 또는 top-p 필터링을 추가
def top_k_filtering(logits, top_k=50):
    values, _ = torch.topk(logits, top_k)
    min_value = values[:, -1].unsqueeze(1)
    return torch.where(logits < min_value, torch.full_like(logits, float('-inf')), logits)

# top-p 샘플링 추가 (nucleus sampling)
def top_p_filtering(logits, top_p=0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
    sorted_indices_to_remove = cumulative_probs > top_p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[0, indices_to_remove] = float('-inf')
    return logits

In [12]:
# 테스트 실행 예시

# def clean_text(text):
#     text = text.replace(" .", ".").replace(" ,", ",").replace(" '", "'")
#     text = text.replace('" "', '"').replace("  ", " ")
#     text = text.replace("= = = = = = =", "")  # 반복 기호 제거
#     return text.strip()

import re
def clean_text(text):
    text = re.sub(r"(= ){3,}", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'"\s*,', ',', text)
    text = re.sub(r'\s*"\s*', '', text)
    text = re.sub(r'\s*\.\s*', '. ', text)
    return text.strip()

start_text = "Write a sentimental blog post about spring travel in Seoul. Mention cherry blossoms, street food, and quiet alleys."

generated = generate_text(model, start_text, word2idx, idx2word, max_length=50)
# print('Generated Text:\n', generated)
print("Generated Text:\n", clean_text(generated))

Generated Text:
 write a sentimental blog post about spring travel in blips mention cherry blips street blips and quiet blips @-@ inch ( ahl , and the blue jackets season. deities did not clear the little rock , he remained the tower building and the gods. the tower building. in the name of the game 's character reila can be very distinct from the blue jackets season
